In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'f:\\pythoncode\\ML\\ML Projects\\Text-Summarization-using-NLP'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluatorConfig:
    root_dir : Path
    data_path : Path
    model_path : Path
    tokenizer_path : Path
    metric_file_name : Path

In [18]:
from text_summarization.constants import *
from text_summarization.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluatorConfig:
        config = self.config.model_evaluation
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluatorConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metrics_file_name
        )

        return model_evaluation_config
    

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_from_disk
from tqdm import tqdm
from evaluate import load  # Ensure you import load from the evaluate library
import pandas as pd

In [16]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluatorConfig):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """Split the dataset into smaller batches for processing."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i: i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                      batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu",
                                      column_text="article",
                                      column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
            inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                               padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                       attention_mask=inputs["attention_mask"].to(device),
                                       length_penalty=0.8, num_beams=8, max_length=128)

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                                 for s in summaries]
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        score = metric.compute()
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        # Load data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_metric = load("rouge")

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size=2,
            column_text='dialogue', column_summary='summary'
        )

        # Modify to directly access the scores if they are floats
        rouge_dict = {rn: score[rn] for rn in ["rouge1", "rouge2", "rougeL", "rougeLsum"]}

        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)



In [21]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2024-10-29 00:54:43,146: INFO: common: Successfully loaded YAML data from config\config.yaml]
[2024-10-29 00:54:43,158: INFO: common: Successfully loaded YAML data from params.yaml]
[2024-10-29 00:54:43,164: INFO: common: Created directory: artifacts]
[2024-10-29 00:54:43,164: INFO: common: Created directory: artifacts/model_evaluation]


100%|██████████| 5/5 [03:00<00:00, 36.05s/it]


[2024-10-29 00:57:49,560: INFO: rouge_scorer: Using default tokenizer.]
